In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ETL pipeline

Get covid cases data in germany. 
Data is taken from here: https://www.arcgis.com/home/item.html?id=f10774f1c63e40168479a1feb6c7ca74

In [4]:
#df_rki = pd.read_csv('https://www.arcgis.com/sharing/rest/content/items/f10774f1c63e40168479a1feb6c7ca74/data')
df_rki = pd.read_csv('data/RKI_COVID19-2.csv')
df_rki['Meldedatum'] = pd.to_datetime(df_rki['Meldedatum'], format='%Y/%m/%d')

## Stats overview

In [5]:
n_cases = df_rki.loc[df_rki['NeuerFall'].isin([0,1])]['AnzahlFall'].sum()
n_cases_new = df_rki.loc[df_rki['NeuerFall'].isin([-1,1])]['AnzahlFall'].sum()
n_deaths = df_rki.loc[df_rki['NeuerTodesfall'].isin([0,1])]['AnzahlTodesfall'].sum()
n_deaths_new = df_rki.loc[df_rki['NeuerTodesfall'].isin([-1,1])]['AnzahlTodesfall'].sum()
n_recovered = df_rki.loc[df_rki['NeuGenesen'].isin([0,1])]['AnzahlGenesen'].sum()
n_recovered_new = df_rki.loc[df_rki['NeuGenesen'].isin([-1,1])]['AnzahlGenesen'].sum()
n_active = n_cases - n_deaths - n_recovered

df_stats = pd.DataFrame({' ':['Total','New Today'],\
                    'Cases':[n_cases, n_cases_new],\
                    'Recovered':[n_recovered, n_recovered_new],\
                    'Deaths':[n_deaths, n_deaths_new],\
                    'Active':[n_active, ' ']})
df_stats.set_index(' ', inplace=True)
n_cases_new

798

In [6]:
df_cases = df_rki.loc[df_rki['NeuerFall'].isin([0,1])].copy()
df_deaths = df_rki.loc[df_rki['NeuerTodesfall'].isin([0,1])].copy()
df_recovered = df_rki.loc[df_rki['NeuGenesen'].isin([0,1])].copy()

In [7]:
df_cases.drop(columns=['AnzahlTodesfall','NeuerTodesfall','AnzahlGenesen','NeuGenesen','Altersgruppe2'], inplace=True)
df_deaths.drop(columns=['AnzahlFall','NeuerFall','NeuGenesen','AnzahlGenesen','Altersgruppe2'], inplace=True)
df_recovered.drop(columns=['AnzahlFall','NeuerFall','AnzahlTodesfall','NeuerTodesfall','Altersgruppe2'], inplace=True)

Data cleaining:
- We'll later add population data to all districts. The data we pull is for all districts expect of the quaters of Berlin. We thus merge the Berlin data to view it as a single city.
- Converting the dates to proper datetime data types
- drop column `Altersgruppe2` since this is not published any more.
- drop column `Datenstand` as this is always the current date

In [8]:
# merge Berlin cases since we currently don't have population data for the individual districts
df_cases.loc[df_cases['IdLandkreis'].isin(np.arange(11000,11013,1)),'IdLandkreis'] = 11000
# convert columns to datetimes
#df_cases['Meldedatum'] = df_cases['Meldedatum'].apply(pd.to_datetime)
#df_cases['Datenstand'] = df_cases['Datenstand'].apply(lambda x: pd.to_datetime(x.split(',')[0]))
#df_cases['Refdatum'] = df_cases['Refdatum'].apply(pd.to_datetime)
# drop irrelevant columns
df_cases.drop(columns = ['Datenstand'], inplace=True)
df_cases.head()

,FID,IdBundesland,Bundesland,Landkreis,Altersgruppe,Geschlecht,AnzahlFall,Meldedatum,IdLandkreis,NeuerFall,Refdatum,IstErkrankungsbeginn
0,6209203,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,2020-03-14,1001,0,2020/03/16 00:00:00,1
1,6209204,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,2020-03-19,1001,0,2020/03/13 00:00:00,1
2,6209205,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,2020-03-19,1001,0,2020/03/16 00:00:00,1
3,6209206,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,2020-03-21,1001,0,2020/03/13 00:00:00,1
4,6209207,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,2020-03-27,1001,0,2020/03/22 00:00:00,1


In [9]:
# one-hot encode age groups
df_cases = pd.concat([df_cases,pd.get_dummies(df_cases['Altersgruppe'], prefix='age')],axis=1).drop(columns='Altersgruppe')
df_cases.head()

,FID,IdBundesland,Bundesland,Landkreis,Geschlecht,AnzahlFall,Meldedatum,IdLandkreis,NeuerFall,Refdatum,IstErkrankungsbeginn,age_A00-A04,age_A05-A14,age_A15-A34,age_A35-A59,age_A60-A79,age_A80+,age_unbekannt
0,6209203,1,Schleswig-Holstein,SK Flensburg,M,1,2020-03-14,1001,0,2020/03/16 00:00:00,1,0,0,1,0,0,0,0
1,6209204,1,Schleswig-Holstein,SK Flensburg,M,1,2020-03-19,1001,0,2020/03/13 00:00:00,1,0,0,1,0,0,0,0
2,6209205,1,Schleswig-Holstein,SK Flensburg,M,1,2020-03-19,1001,0,2020/03/16 00:00:00,1,0,0,1,0,0,0,0
3,6209206,1,Schleswig-Holstein,SK Flensburg,M,1,2020-03-21,1001,0,2020/03/13 00:00:00,1,0,0,1,0,0,0,0
4,6209207,1,Schleswig-Holstein,SK Flensburg,M,1,2020-03-27,1001,0,2020/03/22 00:00:00,1,0,0,1,0,0,0,0


Next, we pull the population data

Data on inhabitants:
https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/04-kreise.html
could be correct one

In [10]:
# needs packae xlrd
df_population = pd.read_excel('./data/04-kreise.xlsx', sheet_name='Kreisfreie Städte u. Landkreise',skiprows=6, skipfooter=16)
df_population.columns = ['IdLandkreis', 'Bezeichnung','Name','NUTS3','area','pop_tot','pop_male','pop_female','pop_per_sqkm2']
df_population.dropna(axis=0, how='any', inplace=True)
# set integer IdLandkreis
df_population['IdLandkreis'] = df_population['IdLandkreis'].astype(int)
df_population.head(10)

,IdLandkreis,Bezeichnung,Name,NUTS3,area,pop_tot,pop_male,pop_female,pop_per_sqkm2
1,1001,Kreisfreie Stadt,"Flensburg, Stadt",DEF01,56.73,89504.0,44599.0,44905.0,1578.0
2,1002,Kreisfreie Stadt,"Kiel, Landeshauptstadt",DEF02,118.65,247548.0,120566.0,126982.0,2086.0
3,1003,Kreisfreie Stadt,"Lübeck, Hansestadt",DEF03,214.19,217198.0,104371.0,112827.0,1014.0
4,1004,Kreisfreie Stadt,"Neumünster, Stadt",DEF04,71.66,79487.0,39241.0,40246.0,1109.0
5,1051,Kreis,Dithmarschen,DEF05,1428.17,133210.0,65720.0,67490.0,93.0
6,1053,Kreis,Herzogtum Lauenburg,DEF06,1263.07,197264.0,96881.0,100383.0,156.0
7,1054,Kreis,Nordfriesland,DEF07,2083.55,165507.0,81099.0,84408.0,79.0
8,1055,Kreis,Ostholstein,DEF08,1393.02,200581.0,96765.0,103816.0,144.0
9,1056,Kreis,Pinneberg,DEF09,664.25,314391.0,154211.0,160180.0,473.0
10,1057,Kreis,Plön,DEF0A,1083.56,128647.0,62532.0,66115.0,119.0


# Rolling

Create a data frame that has the rolling sum of the cases in the past week

In [ ]:
df_cases.copy()[['Meldedatum','AnzahlFall','IdLandkreis']].groupby(['Meldedatum','IdLandkreis']).sum().reset_index()

In [ ]:
#df_cases.groupby('IdLandkreis').rolling('7d').sum()
df_to_roll = df_cases.copy()[['Meldedatum','AnzahlFall','IdLandkreis']].groupby(['Meldedatum','IdLandkreis']).sum().reset_index()
df_to_roll.sort_values(by='Meldedatum',inplace = True)
df_to_roll


In [43]:
#lkr_all - set(df_to_roll.loc[df_to_roll['Meldedatum'] == date ]['IdLandkreis'])

In [44]:
#pd.date_range(df_to_roll['Meldedatum'].min(), df_to_roll['Meldedatum'].max(), freq = '1D')

To compute rolling 7-day totals of new cases, we need to make sure to have one row per day for every administrative district. Thus, we to add rows with zero cases for all days that are missing.

In [72]:
lkr_all = set(df_to_roll['IdLandkreis'].unique())
# sweep over all days in the data set
for date in pd.date_range(df_to_roll['Meldedatum'].min(), df_to_roll['Meldedatum'].max(), freq = '1D'):
    # add zero rows for all districts that didn't report cases on that day
    for id_lkr in lkr_all - set(df_to_roll.loc[df_to_roll['Meldedatum'] == date ]['IdLandkreis']):
        df_to_roll = df_to_roll.append({'Meldedatum':date, 'IdLandkreis': id_lkr, 'AnzahlFall':0}, ignore_index=True)
    print('fixed date',date)        

fixed date 2020-01-28 00:00:00
fixed date 2020-01-29 00:00:00
fixed date 2020-01-30 00:00:00
fixed date 2020-01-31 00:00:00
fixed date 2020-02-01 00:00:00
fixed date 2020-02-02 00:00:00
fixed date 2020-02-03 00:00:00
fixed date 2020-02-04 00:00:00
fixed date 2020-02-05 00:00:00
fixed date 2020-02-06 00:00:00
fixed date 2020-02-07 00:00:00
fixed date 2020-02-08 00:00:00
fixed date 2020-02-09 00:00:00
fixed date 2020-02-10 00:00:00
fixed date 2020-02-11 00:00:00
fixed date 2020-02-12 00:00:00
fixed date 2020-02-13 00:00:00
fixed date 2020-02-14 00:00:00
fixed date 2020-02-15 00:00:00
fixed date 2020-02-16 00:00:00
fixed date 2020-02-17 00:00:00
fixed date 2020-02-18 00:00:00
fixed date 2020-02-19 00:00:00
fixed date 2020-02-20 00:00:00
fixed date 2020-02-21 00:00:00
fixed date 2020-02-22 00:00:00
fixed date 2020-02-23 00:00:00
fixed date 2020-02-24 00:00:00
fixed date 2020-02-25 00:00:00
fixed date 2020-02-26 00:00:00
fixed date 2020-02-27 00:00:00
fixed date 2020-02-28 00:00:00
fixed da

In [73]:
df_to_roll.sort_values(by='Meldedatum',inplace = True)
df_to_roll

,Meldedatum,IdLandkreis,AnzahlFall
0,2020-01-28,5513,2
18999,2020-01-28,16062,0
18998,2020-01-28,16061,0
18997,2020-01-28,16056,0
18996,2020-01-28,16055,0
...,...,...,...
41832,2020-05-11,3101,0
41831,2020-05-11,8222,0
41830,2020-05-11,8221,0
41840,2020-05-11,8231,0


In [74]:
df_to_roll.sort_values(by='Meldedatum',inplace = True)
df_to_roll = df_to_roll.set_index('Meldedatum').groupby('IdLandkreis').rolling('7d').sum()

df_to_roll = df_to_roll.drop(columns = ['IdLandkreis']).reset_index()
#df_to_roll.loc[df_to_roll['IdLandkreis'] == 5558]

In [75]:
df_cases_roll = pd.merge(df_to_roll, df_population,on='IdLandkreis')
df_cases_roll.insert(3,'AnzahlFall100k',0)
df_cases_roll['AnzahlFall100k'] = df_roll['AnzahlFall']/df_roll['pop_tot']*(10**5)
df_cases_roll
df_cases_roll.loc[df_roll['IdLandkreis']==5558].tail(25)

,IdLandkreis,Meldedatum,AnzahlFall,AnzahlFall100k,Bezeichnung,Name,NUTS3,area,pop_tot,pop_male,pop_female,pop_per_sqkm2
9845,5558,2020-04-17,34.0,15.459535,Kreis,Coesfeld,DEA35,1112.05,219929.0,108564.0,111365.0,198.0
9846,5558,2020-04-18,28.0,12.731381,Kreis,Coesfeld,DEA35,1112.05,219929.0,108564.0,111365.0,198.0
9847,5558,2020-04-19,23.0,10.457921,Kreis,Coesfeld,DEA35,1112.05,219929.0,108564.0,111365.0,198.0
9848,5558,2020-04-20,27.0,12.276689,Kreis,Coesfeld,DEA35,1112.05,219929.0,108564.0,111365.0,198.0
9849,5558,2020-04-21,25.0,11.367305,Kreis,Coesfeld,DEA35,1112.05,219929.0,108564.0,111365.0,198.0
9850,5558,2020-04-22,24.0,10.912613,Kreis,Coesfeld,DEA35,1112.05,219929.0,108564.0,111365.0,198.0
9851,5558,2020-04-23,27.0,12.276689,Kreis,Coesfeld,DEA35,1112.05,219929.0,108564.0,111365.0,198.0
9852,5558,2020-04-24,22.0,10.003228,Kreis,Coesfeld,DEA35,1112.05,219929.0,108564.0,111365.0,198.0
9853,5558,2020-04-25,16.0,7.275075,Kreis,Coesfeld,DEA35,1112.05,219929.0,108564.0,111365.0,198.0
9854,5558,2020-04-26,19.0,8.639152,Kreis,Coesfeld,DEA35,1112.05,219929.0,108564.0,111365.0,198.0


In [76]:
df_cases_roll['Landkreis'] = df_cases_roll['IdLandkreis']
# this step is a bit slow, could likely be improved
df_cases_roll['Landkreis'] = df_cases_roll['Landkreis'].apply(lambda x: df_cases.loc[df_cases['IdLandkreis'] == x]['Landkreis'].iloc[0])

In [77]:
df_cases_roll
df_cases_roll.rename(columns={"AnzahlFall":"7d_AnzahlFall",'AnzahlFall100k':'7d_AnzahlFall100k'})

,IdLandkreis,Meldedatum,7d_AnzahlFall,7d_AnzahlFall100k,Bezeichnung,Name,NUTS3,area,pop_tot,pop_male,pop_female,pop_per_sqkm2,Landkreis
0,1001,2020-01-28,0.0,0.000000,Kreisfreie Stadt,"Flensburg, Stadt",DEF01,56.73,89504.0,44599.0,44905.0,1578.0,SK Flensburg
1,1001,2020-01-29,0.0,0.000000,Kreisfreie Stadt,"Flensburg, Stadt",DEF01,56.73,89504.0,44599.0,44905.0,1578.0,SK Flensburg
2,1001,2020-01-30,0.0,0.000000,Kreisfreie Stadt,"Flensburg, Stadt",DEF01,56.73,89504.0,44599.0,44905.0,1578.0,SK Flensburg
3,1001,2020-01-31,0.0,0.000000,Kreisfreie Stadt,"Flensburg, Stadt",DEF01,56.73,89504.0,44599.0,44905.0,1578.0,SK Flensburg
4,1001,2020-02-01,0.0,0.000000,Kreisfreie Stadt,"Flensburg, Stadt",DEF01,56.73,89504.0,44599.0,44905.0,1578.0,SK Flensburg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42100,16077,2020-05-07,2.0,2.219312,Landkreis,Altenburger Land,DEG0M,569.40,90118.0,44138.0,45980.0,158.0,LK Altenburger Land
42101,16077,2020-05-08,2.0,2.219312,Landkreis,Altenburger Land,DEG0M,569.40,90118.0,44138.0,45980.0,158.0,LK Altenburger Land
42102,16077,2020-05-09,2.0,2.219312,Landkreis,Altenburger Land,DEG0M,569.40,90118.0,44138.0,45980.0,158.0,LK Altenburger Land
42103,16077,2020-05-10,2.0,2.219312,Landkreis,Altenburger Land,DEG0M,569.40,90118.0,44138.0,45980.0,158.0,LK Altenburger Land


# Cases overview

In [11]:
df_rki.groupby(['Meldedatum','Bundesland']).sum().reset_index()\
                [['Meldedatum','Bundesland','AnzahlFall','AnzahlTodesfall']]

,Meldedatum,Bundesland,AnzahlFall,AnzahlTodesfall
0,2020-01-28,Bayern,2,0
1,2020-01-28,Nordrhein-Westfalen,2,0
2,2020-01-29,Bayern,2,0
3,2020-01-31,Bayern,3,0
4,2020-02-03,Bayern,1,0
...,...,...,...,...
1139,2020-05-12,Saarland,6,0
1140,2020-05-12,Sachsen,2,0
1141,2020-05-12,Sachsen-Anhalt,2,0
1142,2020-05-12,Schleswig-Holstein,14,0


# Long data format

In [12]:
df_ctr_cases = df_cases.groupby(['Meldedatum'])\
                        .sum()[['AnzahlFall']]
df_ctr_deaths = df_deaths.groupby(['Meldedatum'])\
                        .sum()[['AnzahlTodesfall']]
df_ctr_recovered = df_recovered.groupby(['Meldedatum'])\
                        .sum()[['AnzahlGenesen']]
df_ctr_cases = pd.melt(df_ctr_cases.reset_index(), id_vars=['Meldedatum'],\
                                                   value_vars = ['AnzahlFall'],\
                                                   var_name = 'category',\
                                                   value_name = 'Number')
df_ctr_deaths = pd.melt(df_ctr_deaths.reset_index(), id_vars=['Meldedatum'],\
                                                   value_vars = ['AnzahlTodesfall'],\
                                                   var_name = 'category',\
                                                   value_name = 'Number')
df_ctr_recovered = pd.melt(df_ctr_recovered.reset_index(), id_vars=['Meldedatum'],\
                                                   value_vars = ['AnzahlGenesen'],\
                                                   var_name = 'category',\
                                                   value_name = 'Number')
df_ctr = pd.concat([df_ctr_cases,df_ctr_deaths,df_ctr_recovered], axis = 0)
df_ctr['category'] = df_ctr['category']\
        .apply(lambda x: 'case' if x == 'AnzahlFall' else\
                              ('death' if x == 'AnzahlTodesfall' else 'recovered'))
df_ctr

,Meldedatum,category,Number
0,2020-01-28,case,4
1,2020-01-29,case,2
2,2020-01-31,case,3
3,2020-02-03,case,1
4,2020-02-04,case,4
...,...,...,...
86,2020-05-08,recovered,49
87,2020-05-09,recovered,21
88,2020-05-10,recovered,14
89,2020-05-11,recovered,25


In [13]:
df_ctr_cum = df_ctr.copy().sort_values(by=['Meldedatum','category'])
for el in list(df_ctr_cum['category'].unique()):
    df_ctr_cum.loc[df_ctr_cum['category']== el,'Number' ] = \
        np.cumsum(df_ctr_cum.loc[df_ctr_cum['category']== el,'Number' ])
df_ctr_cum    

,Meldedatum,category,Number
0,2020-01-28,case,4
0,2020-01-28,recovered,4
1,2020-01-29,case,6
1,2020-01-29,recovered,6
2,2020-01-31,case,9
...,...,...,...
67,2020-05-11,death,7631
89,2020-05-11,recovered,148692
90,2020-05-12,case,171306
68,2020-05-12,death,7634


### States

In [14]:
df_sta = pd.concat([df_cases.groupby(['Meldedatum','Bundesland']).sum().reset_index()\
                [['Meldedatum','Bundesland','AnzahlFall']],\
            df_deaths.groupby(['Meldedatum','Bundesland']).sum().reset_index()\
                            [['Meldedatum','Bundesland','AnzahlTodesfall']],\
            df_recovered.groupby(['Meldedatum','Bundesland']).sum().reset_index()\
                            [['Meldedatum','Bundesland','AnzahlGenesen']]])
df_sta = df_sta.fillna(0).groupby(['Meldedatum','Bundesland']).sum().reset_index()
df_sta[['AnzahlFall','AnzahlTodesfall','AnzahlGenesen']] = df_sta[['AnzahlFall','AnzahlTodesfall','AnzahlGenesen']].astype('int64')
df_sta

,Meldedatum,Bundesland,AnzahlFall,AnzahlTodesfall,AnzahlGenesen
0,2020-01-28,Bayern,2,0,2
1,2020-01-28,Nordrhein-Westfalen,2,0,2
2,2020-01-29,Bayern,2,0,2
3,2020-01-31,Bayern,3,0,3
4,2020-02-03,Bayern,1,0,1
...,...,...,...,...,...
1139,2020-05-12,Saarland,6,0,0
1140,2020-05-12,Sachsen,2,0,0
1141,2020-05-12,Sachsen-Anhalt,2,0,0
1142,2020-05-12,Schleswig-Holstein,14,0,0


In [15]:
df_sta_cum = df_sta.copy()
for state in list(df_sta['Bundesland'].unique()):
    for col in ['AnzahlFall','AnzahlTodesfall','AnzahlGenesen']:
        df_sta_cum.loc[df_sta_cum['Bundesland']==state,col] = np.cumsum(df_sta_cum.loc[df_sta_cum['Bundesland']==state,col])
df_sta_cum

,Meldedatum,Bundesland,AnzahlFall,AnzahlTodesfall,AnzahlGenesen
0,2020-01-28,Bayern,2,0,2
1,2020-01-28,Nordrhein-Westfalen,2,0,2
2,2020-01-29,Bayern,4,0,4
3,2020-01-31,Bayern,7,0,7
4,2020-02-03,Bayern,8,0,8
...,...,...,...,...,...
1139,2020-05-12,Saarland,2667,144,2380
1140,2020-05-12,Sachsen,4962,191,4330
1141,2020-05-12,Sachsen-Anhalt,1650,52,1468
1142,2020-05-12,Schleswig-Holstein,2972,125,2546


### Districts

In [16]:
df_recovered.groupby(['Meldedatum','Landkreis']).sum().reset_index()

,Meldedatum,Landkreis,FID,IdBundesland,IdLandkreis,NeuGenesen,AnzahlGenesen,IstErkrankungsbeginn
0,2020-01-28,LK Landsberg a.Lech,6300127,9,9181,0,1,1
1,2020-01-28,LK Starnberg,6303227,9,9188,0,1,1
2,2020-01-28,SK Gelsenkirchen,12481149,10,11026,0,2,1
3,2020-01-29,LK Fürstenfeldbruck,6299208,9,9179,0,1,1
4,2020-01-29,SK München,6292951,9,9162,0,1,1
...,...,...,...,...,...,...,...,...
17306,2020-05-12,LK Freising,6298703,9,9178,1,1,1
17307,2020-05-12,LK Günzburg,6323475,9,9774,1,1,1
17308,2020-05-12,LK Rhein-Erft-Kreis,6235979,5,5362,1,1,1
17309,2020-05-12,SK Duisburg,6224902,5,5112,1,1,1


In [23]:
df_lkr = pd.concat([df_cases.groupby(['Meldedatum','Landkreis']).sum().reset_index()\
                [['Meldedatum','Landkreis','AnzahlFall']],\
            df_deaths.groupby(['Meldedatum','Landkreis']).sum().reset_index()\
                            [['Meldedatum','Landkreis','AnzahlTodesfall']],\
            df_recovered.groupby(['Meldedatum','Landkreis']).sum().reset_index()\
                            [['Meldedatum','Landkreis','AnzahlGenesen']]])
df_lkr = df_lkr.fillna(0).groupby(['Meldedatum','Landkreis']).sum().reset_index()
df_lkr[['AnzahlFall','AnzahlTodesfall','AnzahlGenesen']] = df_lkr[['AnzahlFall','AnzahlTodesfall','AnzahlGenesen']].astype('int64')
df_lkr

,Meldedatum,Landkreis,AnzahlFall,AnzahlTodesfall,AnzahlGenesen
0,2020-01-28,LK Landsberg a.Lech,1,0,1
1,2020-01-28,LK Starnberg,1,0,1
2,2020-01-28,SK Gelsenkirchen,2,0,2
3,2020-01-29,LK Fürstenfeldbruck,1,0,1
4,2020-01-29,SK München,1,0,1
...,...,...,...,...,...
19580,2020-05-12,SK Stuttgart,5,0,0
19581,2020-05-12,SK Weiden i.d.OPf.,2,0,0
19582,2020-05-12,SK Wiesbaden,5,0,0
19583,2020-05-12,SK Würzburg,1,0,0


In [26]:
df_lkr_cum = df_lkr.copy()
for el in list(df_lkr['Landkreis'].unique()):
    for col in ['AnzahlFall','AnzahlTodesfall','AnzahlGenesen']:
        df_lkr_cum.loc[df_lkr_cum['Landkreis']==el,col] = np.cumsum(df_lkr_cum.loc[df_lkr_cum['Landkreis']==el,col])
df_lkr_cum

,Meldedatum,Landkreis,AnzahlFall,AnzahlTodesfall,AnzahlGenesen
0,2020-01-28,LK Landsberg a.Lech,1,0,1
1,2020-01-28,LK Starnberg,1,0,1
2,2020-01-28,SK Gelsenkirchen,2,0,2
3,2020-01-29,LK Fürstenfeldbruck,1,0,1
4,2020-01-29,SK München,1,0,1
...,...,...,...,...,...
19580,2020-05-12,SK Stuttgart,1385,56,1238
19581,2020-05-12,SK Weiden i.d.OPf.,312,18,274
19582,2020-05-12,SK Wiesbaden,359,12,298
19583,2020-05-12,SK Würzburg,466,48,383


## Save Data

In [122]:
df_cases.to_csv('data_cases.csv', index=False)
df_deaths.to_csv('data_deaths.csv', index=False)
df_recovered.to_csv('data_recovered.csv', index=False)
df_stats.to_csv('data_stats.csv')
df_roll.to_csv('data_cases_rolling.csv', index=False)
df_population.to_csv('data_population.csv', index=False)
# country data
df_ctr.to_csv('data_ctr_long.csv', index=False)
df_ctr_cum.to_csv('data_ctr_cum_long.csv', index=False)

In [169]:
# state data
df_sta.to_csv('data_sta_long.csv', index=False)
df_sta_cum.to_csv('data_sta_cum_long.csv', index=False)